# Split the Data 
#### Splitting the date as : 
- 60% Train set 
- 20% Validation set 
- 20% Test Set 

In [ ]:
import pandas as pd 




In [ ]:
def remove_unamed(dfs):
    removed = []
    for df in dfs:
        removed.append(df.loc[:, ~df.columns.str.contains('^Unnamed')])
        


fallen_Trees = 'fallen_Trees'
uncollected_waste = 'uncollected_waste'

def map_distortion(value):
    
    if value == 0:
        return 'allocated_waste'
    
    return 'fallen_trees'
    


In [ ]:
df = pd.read_csv('annotation.csv')

df.columns

In [ ]:
df.class_name.value_counts()

In [ ]:
df.query('class_name == "fallen_Trees"')

In [ ]:
df.class_name = df.class_name.apply(lambda x : uncollected_waste if x == 0 else fallen_Trees)


In [ ]:
df.class_name.unique()

In [ ]:
# df.query('class_name == 0')

In [ ]:
df.shape

In [ ]:
val_set = df.sample(frac=0.2,random_state=1337)
val_set.folder_name = 'VALIDATE'
df.drop(val_set.index,inplace=True)
val_set = val_set.loc[:, ~val_set.columns.str.contains('^Unnamed')]
val_set.shape
val_set.head()
val_set.shape

In [ ]:
test_set = df.sample(frac=0.2,random_state=1337)
test_set.folder_name = 'TEST'
test_set = test_set.loc[:, ~test_set.columns.str.contains('^Unnamed')]
df.drop(test_set.index,inplace=True)
test_set.shape

In [ ]:
train_set = df
train_set.folder_name = 'TRAIN'
train_set = train_set.loc[:, ~train_set.columns.str.contains('^Unnamed')]
train_set.shape


In [ ]:
df = pd.concat([test_set,val_set,train_set])

In [ ]:
df.folder_name.unique()

In [ ]:
df

In [ ]:
df.to_csv('cleared.csv', header=False, index=False)

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf

In [ ]:
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
spec = model_spec.get('efficientdet_lite0')


In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('cleared.csv')


In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

In [ ]:
model.evaluate(test_data)

In [ ]:
model.export(export_dir='.')


In [25]:
model.evaluate_tflite('model.tflite', test_data)

InvalidArgumentError: Incompatible shapes: [1,25] vs. [4] [Op:Mul]